## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

     |████████████████████████████████| 8.9MB 8.6MB/s 
     |████████████████████████████████| 747kB 48.4MB/s 
     |████████████████████████████████| 1.6MB 9.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=94f9b7c9b1623701a28ae628aaf728892593138a6da4bc8101194d9a6ba54641
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
df = pd.read_csv('CustomerReviews.csv')

In [28]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,ม...",1,0.998587
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธรร...",1,0.998240
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,😆,...",0,0.997785
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,25...",1,0.997907
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,%...",0,0.997010


## **Tokenize Words**

In [29]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [30]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [31]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,ม...",1,0.998587
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธรร...",1,0.998240
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,😆,...",0,0.997785
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,25...",1,0.997907
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,%...",0,0.997010


## **Create Dictionary**

In [32]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [33]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', ':', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่าง

In [34]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [35]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 371 ms, sys: 577 µs, total: 371 ms
Wall time: 370 ms


In [36]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.081889 -0.020693       1        1  40.060600
3      0.086342 -0.021902       2        1  26.988841
4      0.004835  0.052246       3        1  12.822692
5     -0.012185 -0.001295       4        1   7.616820
0     -0.006976  0.004422       5        1   7.194321
2      0.009796 -0.004116       6        1   5.291702
1      0.000079 -0.008663       7        1   0.025024, topic_info=        Term       Freq      Total Category  logprob  loglift
17       น้ำ  25.000000  25.000000  Default  30.0000  30.0000
117      ทาน  23.000000  23.000000  Default  29.0000  29.0000
57      ชาบู  19.000000  19.000000  Default  28.0000  28.0000
26      ราคา  16.000000  16.000000  Default  27.0000  27.0000
37     เนื้อ  18.000000  18.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
134       สด   0.001959   9.069233   Topic7  -5.1999  -0.1471
18   น้ำจิ้ม   0.001488   6.329609   Topic7  -5.4749  -0.0624
136     สั่ง   0.001767   9.133052   Topic7  -5.3030  -0.2572
91         :   0.001905  11.803276   Topic7  -5.2282  -0.4388
337        -   0.001882  11.636953   Topic7  -5.2401  -0.4366

[475 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
86        1  0.848759       
336       2  0.595612      "
336       5  0.297806      "
337       1  0.085933      -
337       2  0.601532      -
...     ...       ...    ...
168       1  0.927590    ไม้
181       3  1.031751     ไว
47        1  0.540806  ไอติม
47        2  0.270403  ไอติม
48        1  0.880349      ​

[384 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 5, 6, 1, 3, 2])

In [37]:
model.show_topic(3)

[('-', 0.01925246),
 ('กิน', 0.018865569),
 ('กุ้ง', 0.01873097),
 ('ทาน', 0.015803384),
 ('ดี', 0.014061399),
 ('สด', 0.013119288),
 ('อาหาร', 0.012377823),
 ('ชาบู', 0.011982249),
 ('เนื้อ', 0.011257381),
 ('คน', 0.010598079)]

In [38]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [39]:
df[['Restaurant', 'Review', 'topics']]

,Restaurant,Review,topics
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,6
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,6
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,6
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,6
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,6
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,5
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,2
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,3
